In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import sys
sys.path.append("/home/mcamara/taxi_demand_predictor")

In [24]:
import src.config as config

In [25]:
from datetime import datetime

In [26]:
import pandas as pd

In [27]:
current_date = pd.to_datetime(datetime.utcnow()).floor('H')

In [28]:
from src.feature_store_api import get_feature_group

In [29]:
predictions_fg = get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTIONS)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/24729
Connected. Call `.close()` to terminate connection gracefully.


In [30]:
actuals_fg = get_feature_group(name=config.FEATURE_GROUP_NAME)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/24729
Connected. Call `.close()` to terminate connection gracefully.


In [31]:
from datetime import timedelta

In [32]:
query = predictions_fg.select_all().join(actuals_fg.select_all(), 
                                         on=['pickup_hour', 'pickup_location_id']).filter(predictions_fg.pickup_hour >= current_date - timedelta(days=30))

In [33]:
from src.feature_store_api import get_feature_store

In [34]:
feature_store = get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/24729
Connected. Call `.close()` to terminate connection gracefully.


In [35]:
try:
    # create feature view as it does not exist yet\n",
    feature_store.create_feature_view( 
    name=config.FEATURE_VIEW_MONITORING, 
    version=1, 
    query=query 
    ) 
except: 
    print('Feature view already existed. Skip creation.') 
     

Feature view already existed. Skip creation.


In [36]:
predictions_and_actuals_fv = feature_store.get_feature_view(name=config.FEATURE_VIEW_MONITORING, 
                                                            version=1) 

In [37]:
preds_and_actuals = predictions_and_actuals_fv.get_batch_data(start_time=(current_date - timedelta(days=30)),
                                                              end_time=current_date)
   

2023-03-29 15:36:15,033 INFO: USE `taxi_demand_mc_featurestore`
2023-03-29 15:36:15,496 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`pickup_location_id` `pickup_location_id`, `fg1`.`predicted_demand` `predicted_demand`, `fg1`.`pickup_hour` `pickup_hour`, `fg1`.`pickup_location_id` `join_pk_pickup_location_id`, `fg1`.`pickup_hour` `join_pk_pickup_hour`, `fg1`.`pickup_hour` `join_evt_pickup_hour`, `fg0`.`rides` `rides`, RANK() OVER (PARTITION BY `fg1`.`pickup_location_id`, `fg1`.`pickup_hour`, `fg1`.`pickup_hour` ORDER BY `fg0`.`pickup_hour` DESC) pit_rank_hopsworks
FROM `taxi_demand_mc_featurestore`.`model_predictions_feature_group_1` `fg1`
INNER JOIN `taxi_demand_mc_featurestore`.`time_series_hourly_feature_group_1` `fg0` ON `fg1`.`pickup_location_id` = `fg0`.`pickup_location_id` AND `fg1`.`pickup_hour` = `fg0`.`pickup_hour` AND `fg1`.`pickup_hour` >= `fg0`.`pickup_hour`
WHERE `fg1`.`pickup_hour` >= TIMESTAMP '2023-02-26 15:00:00.000' AND `fg1`.`pickup_hour` >= TIMESTAMP '2023

In [38]:
preds_and_actuals = preds_and_actuals[preds_and_actuals.pickup_hour.between(current_date - timedelta(days=30),
                                                                            current_date)]

In [39]:
df = preds_and_actuals

In [40]:
from sklearn.metrics import mean_absolute_error

In [41]:
mae = mean_absolute_error(df['rides'], df['predicted_demand'])
print(f'{mae=:.2f}')

mae=9.81
